In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [121]:
data = pd.read_excel('CTG.xls', 'Raw Data')
data.head()




description = pd.read_excel('CTG.xls', 'Description', skiprows=5)
description.columns



description.head()
columnNames = description
columnNames = description[['Unnamed: 2', 'Unnamed: 3']]
columnNames = columnNames[:39]
columnNames = columnNames.T
columnNames # Obtained a columName df
data.head()
#data = data.drop('SegFile', 1) # Removed the column that wasn't common
data.head()


columnNames.T
data.head()

,FileName,Date,SegFile,b,e,LBE,LB,AC,FM,UC,...,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Variab10.txt,1996-12-01,CTG0001.txt,240.0,357.0,120.0,120.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9.0,2.0
2,Fmcs_1.txt,1996-05-03,CTG0002.txt,5.0,632.0,132.0,132.0,4.0,0.0,4.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
3,Fmcs_1.txt,1996-05-03,CTG0003.txt,177.0,779.0,133.0,133.0,2.0,0.0,5.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0
4,Fmcs_1.txt,1996-05-03,CTG0004.txt,411.0,1192.0,134.0,134.0,2.0,0.0,6.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,6.0,1.0


In [62]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
b,2126.0,878.439793,894.084748,0.0,55.0,538.0,1521.00,3296.0
e,2126.0,1702.877234,930.919143,287.0,1009.0,1241.0,2434.75,3599.0
LBE,2126.0,133.303857,9.840844,106.0,126.0,133.0,140.00,160.0
LB,2126.0,133.303857,9.840844,106.0,126.0,133.0,140.00,160.0
AC,2126.0,2.722484,3.560850,0.0,0.0,1.0,4.00,26.0
FM,2127.0,7.503056,39.030452,0.0,0.0,0.0,2.00,564.0
UC,2127.0,3.669017,2.877148,0.0,1.0,3.0,5.00,23.0
ASTV,2127.0,47.008933,17.210648,12.0,32.0,49.0,61.00,87.0
MSTV,2127.0,1.335449,0.891543,0.2,0.7,1.2,1.70,7.0
ALTV,2127.0,9.884814,18.476534,0.0,0.0,0.0,11.00,91.0


In [118]:

# data = data.drop(data.NSP == np.nan ) # dropped NaN columns from NSP column

# data = data.replace('', np.nan,inplace=True) # Replacing any blank with NaN


# df1 = df[df.isnull().any(axis=1)] # Check to see if any of the cells contains a NaN cell


#Our Dataframe is now NaN free


In [122]:
data.columns

Index(['FileName', 'Date', 'SegFile', 'b', 'e', 'LBE', 'LB', 'AC', 'FM', 'UC',
       'ASTV', 'MSTV', 'ALTV', 'MLTV', 'DL', 'DS', 'DP', 'DR', 'Width', 'Min',
       'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean', 'Median', 'Variance',
       'Tendency', 'A', 'B', 'C', 'D', 'E', 'AD', 'DE', 'LD', 'FS', 'SUSP',
       'CLASS', 'NSP'],
      dtype='object')

In [123]:
subset = data[['SegFile','A','B','C','D','E','AD','DE','LD','FS','SUSP','CLASS','NSP']]

subset_cols = ['A','B','C','D','E','AD','DE','LD','FS','SUSP','CLASS','NSP']


subset = subset.dropna()

In [136]:
#checking if there are duplicate classifications among segFiles
#Ideally each cell should return the value 1
all_cols_unique_classification = subset.groupby('SegFile').agg({col:'nunique' for col in subset})
all_cols_unique_classification

,C,AD,B,DE,A,NSP,FS,CLASS,E,D,SUSP,SegFile,LD
SegFile,,,,,,,,,,,,,
CTG0001.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0002.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0003.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0004.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0005.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0006.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0007.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0008.txt,1,1,1,1,1,1,1,1,1,1,1,1,1
CTG0009.txt,1,1,1,1,1,1,1,1,1,1,1,1,1


In [137]:

# Checking if any cell returns value greater than 1
all_cols_unique_classification[(all_cols_unique_classification>1).any(axis=1)]

,C,AD,B,DE,A,NSP,FS,CLASS,E,D,SUSP,SegFile,LD
SegFile,,,,,,,,,,,,,


In [149]:
subset_cols = ['A','B','C','D','E','AD','DE','LD','FS','SUSP','CLASS','NSP']

subset = subset[['A','B','C','D','E','AD','DE','LD','FS','SUSP','CLASS','NSP']].astype(int)
subset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 1 to 2126
Data columns (total 12 columns):
A        2126 non-null int32
B        2126 non-null int32
C        2126 non-null int32
D        2126 non-null int32
E        2126 non-null int32
AD       2126 non-null int32
DE       2126 non-null int32
LD       2126 non-null int32
FS       2126 non-null int32
SUSP     2126 non-null int32
CLASS    2126 non-null int32
NSP      2126 non-null int32
dtypes: int32(12)
memory usage: 116.3 KB


In [152]:
subset.head()
# sns.set(color_codes=True)
# sns.distplot(subset)

,A,B,C,D,E,AD,DE,LD,FS,SUSP,CLASS,NSP
1,0,0,0,0,0,0,0,0,1,0,9,2
2,0,0,0,0,0,1,0,0,0,0,6,1
3,0,0,0,0,0,1,0,0,0,0,6,1
4,0,0,0,0,0,1,0,0,0,0,6,1
5,0,1,0,0,0,0,0,0,0,0,2,1
